## New Built-In Projections

ipyleaflet now supports custom map projections and includes 2 base layers for polar projections:
NASA's Next GenerationBlue Marble 500m for the Arctic and Antarctic regions.

In [1]:
import geopandas, os, json, requests
from ipyleaflet import (
    Map,
    basemaps,
    basemap_to_tiles,
    Marker,
    TileLayer,
    WMSLayer,
    Polygon,
    GeoJSON,
    GeoData,
    DrawControl,
    projections
)

def handle_draw(target, action, geo_json):
    print(action, geo_json)
    
dc = DrawControl(marker={'shapeOptions': {'color': '#0000FF'}})
dc.on_draw(handle_draw)

# note that we need to use the same projection for the our layer and the map.
m1 = Map(center=(90, 0),
        zoom=0,
        basemap=basemaps.NASAGIBS.BlueMarble3413,
        crs=projections.EPSG3413)
m1.add_control(dc)
m1

Map(center=[90, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

### Data Layers with custom projections

If we want to use a vector layer with a custom projection Leaflet will do the reprojection as long as our layer uses WS84 (lat lon) coordinates. This is particularly handy with GeoJSON as we don't need to do anything to reproject our features. The following cell will load the european contries boundaries and will reproject them to be render in our polar projection. 



In [4]:
def download_geojson(projection_sample, url):
    if not os.path.exists(projection_sample):
        url = url
        r = requests.get(url)
        with open('europe_110.geo.json', 'w') as f:
            f.write(r.content.decode("utf-8"))

    with open('europe_110.geo.json', 'r') as f:
        data = json.load(f)
    return data

geo_json = download_geojson('europe_110.geo.json',
                            'https://github.com/jupyter-widgets/ipyleaflet/raw/master/examples/europe_110.geo.json')

geo_data = GeoJSON(data = geo_json,
                   style={'color': 'black', 'fillColor': '#366370', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': '#b08a3e' , 'fillOpacity': 0.9},
                   name = 'Countries')
m1.add_layer(geo_data)

## Custom Projections

Chances are that if you are reading this you want to use a custom projection for your map. This is now supported via [Proj4Leaflet](https://github.com/kartena/Proj4Leaflet). The only thing you need to do is to declare the properties of your projection and Leaflet will do the rest. 

Keep in mind that this feature is for predefined projections, so if you use a backend as GeoServer, MapServer etc, they can do the reprojection for you and you can use that projection to show your layers in a map. **ipyleaflet won't reproject your layer on the fly**

In [ ]:
# We're going to use https://georepository.com/crs_2163/US-National-Atlas-Equal-Area.html
my_projection = {
    'name': 'EPSG:2163',
    'custom': True, #This is important, it tells ipyleaflet that this projection is not on the predefined ones.
    'proj4def': '+proj=laea +lat_0=45 +lon_0=-100 +x_0=0 +y_0=0 +a=6370997 +b=6370997 +units=m +no_defs',
    'origin': [-2215235.82, 420644.41],
    'bounds': [
        [-8046094.81, 1819060.18],
        [988364.71, 3511186.72]   
    ],
    'resolutions': [
        8192.0,
        4096.0,
        2048.0,
        1024.0,
        512.0,
        256.0
    ]
}

wms = WMSLayer(
    url='https://ahocevar.com/geoserver/wms',
    layers='ne:NE1_HR_LC_SR_W_DR',
    format='image/png',
    transparent=True,
    min_zoom=0,
    attribution='ahocevar geospatial',
    crs=my_projection # I'm asking this WMS service to reproject the tile layer using EPSG:2163
)

m2 = Map(center=(40, -104),
        zoom=0,
        layers=(wms,),
        crs=my_projection)


dc2 = DrawControl(marker={'shapeOptions': {'color': '#0000FF'}})
dc2.on_draw(handle_draw)

m2.add_control(dc2)

m2

## GeoJSON in custom projections.


In [ ]:
import matplotlib
from matplotlib import cm

cmap = cm.get_cmap('viridis')

geo_json = download_geojson('population_diversity_usa.json',
                            'https://www.chapelhillopendata.org/api/v2/catalog/datasets/census-diversity/exports/geojson')

for feature in geo_json['features']:
    diversity_index = float(feature['properties']['divindx_cy']) / 100.0
    rgba = cmap(diversity_index)
    color = f'rgb({rgba[0]},{rgba[1]},{rgba[2]})'
    feature['properties']['style'] = {
        'color': color,
        'weight': 1,
        'fillColor': color,
        'fillOpacity': 0.8
    }

geo_data = GeoJSON(data = geo_json,
                   hover_style={'fillColor': '#b08a3e' , 'fillOpacity': 0.9},
                   name = 'Diversity Index')

m2.add_layer(geo_data)